In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!kaggle datasets list -s cicids

In [ ]:
!kaggle datasets download -d ericanacletoribeiro/cicids2017-cleaned-and-preprocessed




In [ ]:
!unzip cicids2017-cleaned-and-preprocessed.zip

In [ ]:
import pandas as pd

df = pd.read_csv("cicids2017_cleaned.csv")

df.head()

In [ ]:
print("Rows and Columns:",df.shape)

In [ ]:
print("Column Names:\n",df.columns)

In [ ]:
df.describe()


In [ ]:
import numpy as np

df_cleaned = df.copy()

numeric_cols = df_cleaned.select_dtypes(include=[np.number]).columns

df_cleaned[numeric_cols] = df_cleaned[numeric_cols].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
print("Any negatives left?\n", (df_cleaned[numeric_cols] < 0).sum().sum())

In [ ]:
missing_values = df_cleaned.isnull().sum()
print("Missing Values:\n", missing_values)

missing_values = missing_values[missing_values > 0]
print("\nColumns with Missing Values:\n", missing_values)

In [ ]:
from sklearn.preprocessing import LabelEncoder

#create a labelencoder object to convert strings to numbers
encoder = LabelEncoder()

#encode the attack type column
df_cleaned['Attack type'] = encoder.fit_transform(df_cleaned['Attack Type'])

#see what the encoder did
print("Label mapping:", dict(zip(encoder.classes_, encoder.transform(encoder.classes_))))

In [ ]:
from sklearn.model_selection import train_test_split

#split the data into training and testing sets

x = df_cleaned.drop(['Attack Type', 'Attack type'], axis=1)
y = df_cleaned['Attack type']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)# This will print the number of rows and columns in the training and testing sets

print("Training Features (X_train):", X_train.shape)  # (rows, columns)
print("Testing Features (X_test):", X_test.shape)

print("Training Labels (y_train):", y_train.shape)
print("Testing Labels (y_test):", y_test.shape)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)

print("Accuracy:", accuracy)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



In [ ]:
import pandas as pd

# Load the cleaned dataset
df = pd.read_csv('/content/drive/MyDrive/df_cleaned.csv')

# Check the first few rows
df.head()


In [ ]:
#  Sample 2000 rows from each available attack type
df_sampled = df.groupby('Attack Type').apply(lambda x: x.sample(n=2000, random_state=42)).reset_index(drop=True)

#  Check shape (should be 5 classes × 2000 = 10,000 rows)
print("Sampled data shape:", df_sampled.shape)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create a label encoder
label_encoder = LabelEncoder()

# Transform "Attack Type" text labels into numeric values
df_sampled['Label'] = label_encoder.fit_transform(df_sampled['Attack Type'])

# Let's see the mapping
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label mapping:", label_mapping)


In [ ]:
#  Drop the original text label (we already converted it)
X = df_sampled.drop(['Attack Type', 'Label'], axis=1)
y = df_sampled['Label']


In [ ]:
from sklearn.model_selection import train_test_split

# 80% for training, 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# to Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

#  Train the model
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Show evaluation results
print("Classification Report:\n", classification_report(y_test, predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test, predictions))

In [ ]:
import joblib

# Save the model
joblib.dump(model, '/content/drive/MyDrive/random_forest_nids_model.pkl')



In [ ]:
# Sample prediction
sample = X_test.iloc[0].values.reshape(1, -1)
predicted_class = model.predict(sample)[0]

# Decode label
label_map = {0: 'Bots', 1: 'Brute Force', 2: 'DDoS', 3: 'DoS', 4: 'Normal Traffic', 5: 'Port Scanning', 6: 'Web Attacks'}
print("Predicted:", label_map[predicted_class])


In [ ]:
sample = X_test.iloc[[0]]  # Double brackets keep it as a DataFrame
predicted_class = model.predict(sample)[0]

label_map = {0: 'Bots', 1: 'Brute Force', 2: 'DDoS', 3: 'DoS', 4: 'Normal Traffic', 5: 'Port Scanning', 6: 'Web Attacks'}
print("Predicted:", label_map[predicted_class])


In [ ]:
import joblib

# Save the model as a file
joblib.dump(model, '/content/drive/MyDrive/nids_model.pkl')


In [ ]:
joblib.dump(label_encoder, 'label_encoder.pkl')


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import joblib

# Load your cleaned dataset
df = pd.read_csv('/content/drive/MyDrive/cleaned.csv')

# Recreate the encoder
encoder = LabelEncoder()
encoder.fit(df['Attack Type'])

# Save it
joblib.dump(encoder, '/content/drive/MyDrive/label_encoder.pkl')


In [ ]:
print(df_cleaned.columns.tolist())
print(df_cleaned.shape[1])  # Number of features
